In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None,
                         labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers.elmo_indexer import ELMoCharacterMapper, ELMoTokenCharactersIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = ELMoTokenCharactersIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:01, 78.76it/s]
251it [00:00, 260.17it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.elmo_indexer.ELMoTokenCharactersIndexer at 0x1a1f331860>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

We don't need to build the vocab: all that is handled by the token indexer

In [17]:
vocab = Vocabulary()

### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[[259,  66, 111,  ..., 261, 261, 261],
           [259,  45, 260,  ..., 261, 261, 261],
           [259, 112, 103,  ..., 261, 261, 261],
           ...,
           [259, 117, 105,  ..., 261, 261, 261],
           [259, 100, 112,  ..., 261, 261, 261],
           [259, 101, 106,  ..., 261, 261, 261]],
  
          [[259,  35, 260,  ..., 261, 261, 261],
           [259,  85, 105,  ..., 261, 261, 261],
           [259, 112, 115,  ..., 261, 261, 261],
           ...,
           [259,  47, 260,  ..., 261, 261, 261],
           [259,  68, 109,  ..., 261, 261, 261],
           [259,  99, 102,  ..., 261, 261, 261]],
  
          [[259,  35, 260,  ..., 261, 261, 261],
           [259,  35, 260,  ..., 261, 261, 261],
           [259,  35, 260,  ..., 261, 261, 261],
           ...,
           [259,  99, 102,  ..., 261, 261, 261],
           [259, 100,  98,  ..., 261, 261, 261],
           [259,  98, 260,  ..., 261, 261, 261]],
  
          ...,
  
          [[259,  35

In [23]:
batch["tokens"]["tokens"]

tensor([[[259,  66, 111,  ..., 261, 261, 261],
         [259,  45, 260,  ..., 261, 261, 261],
         [259, 112, 103,  ..., 261, 261, 261],
         ...,
         [259, 117, 105,  ..., 261, 261, 261],
         [259, 100, 112,  ..., 261, 261, 261],
         [259, 101, 106,  ..., 261, 261, 261]],

        [[259,  35, 260,  ..., 261, 261, 261],
         [259,  85, 105,  ..., 261, 261, 261],
         [259, 112, 115,  ..., 261, 261, 261],
         ...,
         [259,  47, 260,  ..., 261, 261, 261],
         [259,  68, 109,  ..., 261, 261, 261],
         [259,  99, 102,  ..., 261, 261, 261]],

        [[259,  35, 260,  ..., 261, 261, 261],
         [259,  35, 260,  ..., 261, 261, 261],
         [259,  35, 260,  ..., 261, 261, 261],
         ...,
         [259,  99, 102,  ..., 261, 261, 261],
         [259, 100,  98,  ..., 261, 261, 261],
         [259,  98, 260,  ..., 261, 261, 261]],

        ...,

        [[259,  35, 260,  ..., 261, 261, 261],
         [259,  62, 260,  ..., 261, 261, 261]

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([11, 100, 50])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import ElmoTokenEmbedder

options_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json'
weight_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5'

elmo_embedder = ElmoTokenEmbedder(options_file, weight_file)
word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder})

02/07/2019 17:36:11 - INFO - allennlp.modules.elmo -   Initializing ELMo


In [28]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(), config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[[259,  66, 111,  ..., 261, 261, 261],
          [259,  45, 260,  ..., 261, 261, 261],
          [259, 112, 103,  ..., 261, 261, 261],
          ...,
          [259, 117, 105,  ..., 261, 261, 261],
          [259, 100, 112,  ..., 261, 261, 261],
          [259, 101, 106,  ..., 261, 261, 261]],
 
         [[259,  35, 260,  ..., 261, 261, 261],
          [259,  85, 105,  ..., 261, 261, 261],
          [259, 112, 115,  ..., 261, 261, 261],
          ...,
          [259,  47, 260,  ..., 261, 261, 261],
          [259,  68, 109,  ..., 261, 261, 261],
          [259,  99, 102,  ..., 261, 261, 261]],
 
         [[259,  35, 260,  ..., 261, 261, 261],
          [259,  35, 260,  ..., 261, 261, 261],
          [259,  35, 260,  ..., 261, 261, 261],
          ...,
          [259,  99, 102,  ..., 261, 261, 261],
          [259, 100,  98,  ..., 261, 261, 261],
          [259,  98, 260,  ..., 261, 261, 261]],
 
         ...,
 
         [[259,  35, 260,  ..., 261, 261, 261],
        

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 0.0858,  0.0223, -0.1586,  0.1349, -0.0376, -0.0261],
        [-0.1301,  0.0480,  0.0867,  0.0141,  0.2163, -0.1783],
        [-0.0898,  0.0805,  0.1165, -0.1205, -0.0798, -0.0552],
        [-0.1485,  0.0437,  0.3471, -0.0798,  0.0412, -0.0691],
        [ 0.1237,  0.1616,  0.1916, -0.1183,  0.0581, -0.1586],
        [ 0.2513,  0.0618,  0.1800,  0.1422,  0.3283, -0.1055],
        [ 0.0311, -0.0209, -0.1196, -0.0920, -0.0282, -0.2505],
        [-0.1946,  0.1308,  0.0727, -0.0361, -0.1095, -0.1665],
        [-0.1263, -0.0665,  0.0816, -0.1481,  0.1190, -0.1457],
        [-0.0762, -0.0154,  0.0472,  0.0230, -0.1451, -0.1299],
        [-0.2562,  0.1345,  0.2318,  0.1293,  0.1449, -0.2039]],
       grad_fn=<AddmmBackward>)

In [36]:
model(**batch)

{'class_logits': tensor([[ 0.0389, -0.1613, -0.1306,  0.0497, -0.1056,  0.0422],
         [-0.0693, -0.1457,  0.1586, -0.0689,  0.0846, -0.1528],
         [-0.0561,  0.1929,  0.0745, -0.0527,  0.0945, -0.1080],
         [-0.2267,  0.1510,  0.2946,  0.0169, -0.0651, -0.1007],
         [ 0.0745,  0.0488,  0.2536, -0.1912,  0.0134, -0.0950],
         [ 0.0587,  0.2195, -0.0123,  0.1830,  0.0590, -0.0560],
         [ 0.0060, -0.1080, -0.1402, -0.1042,  0.0381, -0.1645],
         [-0.1112,  0.1329,  0.1719,  0.1552, -0.0425, -0.1649],
         [-0.1144,  0.2286,  0.1458, -0.1806,  0.1630, -0.2000],
         [-0.0798, -0.0302, -0.0057,  0.0076,  0.0402, -0.0796],
         [-0.1634,  0.1240,  0.1739,  0.1157,  0.0166, -0.0234]],
        grad_fn=<AddmmBackward>),
 'loss': tensor(0.6955, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)}

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.6934, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

In [40]:
[x.grad for x in list(model.encoder.parameters())]

[tensor([[-9.2858e-05, -4.9361e-05, -3.7288e-05,  ...,  3.5947e-06,
           6.7686e-06,  5.7458e-05],
         [-1.0111e-04,  1.9700e-04,  2.2716e-05,  ..., -1.1300e-04,
          -2.2771e-05, -3.8708e-04],
         [ 1.7676e-06,  1.4282e-06,  6.0405e-05,  ..., -3.9052e-05,
           1.6350e-04,  1.5471e-04],
         ...,
         [-4.0847e-05,  4.4254e-05, -7.2378e-05,  ...,  2.0154e-04,
           8.3410e-05, -1.2389e-04],
         [-5.1695e-06, -4.6811e-06, -5.3316e-05,  ...,  1.1514e-05,
          -1.1821e-04, -1.1049e-06],
         [ 9.2464e-05,  4.6357e-05, -6.5559e-05,  ..., -1.3204e-04,
          -2.1428e-04, -1.9575e-04]]),
 tensor([[-2.1633e-05,  8.7957e-07, -3.4178e-05,  ..., -9.0936e-06,
           4.9037e-05,  1.9673e-05],
         [ 4.6168e-05,  4.7835e-05,  6.2958e-05,  ..., -5.4513e-05,
          -2.6419e-05,  1.8046e-06],
         [ 2.0971e-05,  6.7690e-05, -7.4084e-05,  ..., -1.8694e-06,
          -4.2899e-06,  1.2227e-05],
         ...,
         [-4.7440e-05, -1

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

02/07/2019 17:36:24 - INFO - allennlp.training.trainer -   Beginning training.
02/07/2019 17:36:24 - INFO - allennlp.training.trainer -   Epoch 0/1
02/07/2019 17:36:24 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 1255.145472
02/07/2019 17:36:24 - INFO - allennlp.training.trainer -   Training
loss: 0.6855 ||: 100%|██████████| 5/5 [00:36<00:00,  8.01s/it]
02/07/2019 17:37:01 - INFO - allennlp.training.trainer -                     Training |  Validation
02/07/2019 17:37:01 - INFO - allennlp.training.trainer -   loss          |     0.686  |       N/A
02/07/2019 17:37:01 - INFO - allennlp.training.trainer -   cpu_memory_MB |  1255.145  |       N/A
02/07/2019 17:37:01 - INFO - allennlp.training.trainer -   Epoch duration: 00:00:36
02/07/2019 17:37:01 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:00:36
02/07/2019 17:37:01 - INFO - allennlp.training.trainer -   Epoch 1/1
02/07/2019 17:37:01 - INFO - allennlp.training.trainer -   Peak CPU memor

# Generating Predictions

In [44]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [45]:
from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [46]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds) 

100%|██████████| 4/4 [00:50<00:00, 13.33s/it]
